### pip install & import

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

In [ ]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

# from xgboost import XGBRFRegressor
from sklearn.model_selection import cross_validate

### 최종데이터 불러오기

In [ ]:
# movie_final_회귀 = pd.read_csv('../3_merge_top300/movie_final_reg.csv',encoding='utf-8')
movie_final_분류 = pd.read_csv('../3_merge_top300/movie_final_cls.csv',encoding='utf-8')

## 원핫인코딩 

In [ ]:
# 원핫인코딩

# movie_labels = pd.get_dummies(movie_final,columns = ['국적','장르','등급','계절'])

In [ ]:
# 독립 변수와 종속 변수 나눔

# 독립 = movie_DL[['감독_흥행', '배급사_흥행', '국적_기타', '국적_독일', '국적_미국',
#        '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF', '장르_가족', '장르_공연',
#        '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬',
#        '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_성인물(에로)', '장르_스릴러',
#        '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지',
#        '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가', '계절_가을', '계절_겨울',
#        '계절_봄', '계절_여름']] 
# 종속 = movie_DL[['전국 관객수']] 

# 스케일링

In [ ]:
from sklearn.preprocessing import RobustScaler

# 객체생성
robust_scaler = RobustScaler()

# *** 만약 특정 열의 스케일링을 하고 싶은 경우 ***
A_n = robust_scaler.fit_transform(movie_final_분류['전국 스크린수'].values.reshape(-1,1))
movie_final_분류.insert(0, '전국 스크린수_scaled', A_n)
# movie_final_분류.drop(['전국 스크린수'], axis=1, inplace=False)

A_n1 = robust_scaler.fit_transform(movie_final_분류['총 관객수'].values.reshape(-1,1))
movie_final_분류.insert(0, '총 관객수_scaled', A_n1)
# movie_final_분류.drop(['총 관객수'], axis=1, inplace=False)

In [ ]:
movie_final_분류.columns

## 라벨링

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_form = LabelEncoder()
le_nation = LabelEncoder()
le_genre = LabelEncoder()
le_rating = LabelEncoder()
le_class = LabelEncoder()
le_season = LabelEncoder()

movie_labels = movie_final_분류.copy()
movie_labels['영화형태'] = le_form.fit_transform(movie_final_분류['영화형태'])
movie_labels['국적'] = le_nation.fit_transform(movie_final_분류['국적'])
movie_labels['장르'] = le_genre.fit_transform(movie_final_분류['장르'])
movie_labels['등급'] = le_rating.fit_transform(movie_final_분류['등급'])
movie_labels['영화구분'] = le_class.fit_transform(movie_final_분류['영화구분'])
movie_labels['계절'] = le_season.fit_transform(movie_final_분류['계절'])
movie_labels.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_labels.head(3)


In [ ]:
# movie_labels[movie_labels['연도']!=2023]

### 데이터분배
1) movie_labels_etc : train과 val data   
2) movie_labels_2023 : test data

In [ ]:
# movie_labels_2023 = movie_labels[movie_labels['연도']==2023]
# movie_labels_etc = movie_labels[movie_labels['연도']!=2023]
# display(movie_labels_etc.head(),
# movie_labels_2023.head())

### 변수 정리   
1. 사용하지 않는 컬럼 제거
2. 독립변수와 종속변수 설정

In [ ]:
# # 사용하지 않는 컬럼 제거

# not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

# movie_DL = movie_labels_etc.drop(not_used, axis=1)      # 2003~2022 데이터 main
# movie_DL2 = movie_labels_2023.drop(not_used, axis=1)    # 2023년 test


In [ ]:
# 사용하지 않는 컬럼 제거

not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

movie_DL = movie_labels.drop(not_used, axis=1)
movie_DL.head()

In [ ]:
# 국적 => 국내 vs 해외
# 관객비율.. 2023년 데이터 총 관객수 몰라....
# 비율 말고 총 관객수를 쓰자
# 총관객수 defqult를 2022년으로 해서 2023년 총관객수 채우면 어때?

In [ ]:
# # 독립 변수와 종속 변수 나눔

# 종속 = movie_DL[['전국 관객수']].copy()
# 종속['전국 관객수'] = 종속['전국 관객수'] / 1000 

# 종속_2023 = movie_DL2[['전국 관객수']].copy()
# 종속_2023['전국 관객수'] = 종속_2023['전국 관객수'] / 1000 


# 독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_scaled',
#                '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

# 독립_2023 = movie_DL2[['장르', '등급', '감독_흥행','총 관객수_scaled',
#                '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

# display(독립.head(), 독립_2023.head())

In [ ]:
독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_scaled',
               '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_scaled','연도','코로나']].copy()

종속 = movie_DL[['전국 관객수']].copy()

display(독립.info(), 종속.info())

## train, test 데이터 나누기

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(독립, 종속, test_size=0.20)

In [ ]:
train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.20)
print(len(train_input), len(val_input), len(test_input))

In [ ]:
# train, validation 데이터 나눔

from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(독립,종속,test_size=0.25,random_state=42)

## target 클래스 분류하기

In [ ]:
display(
    len(train_target[train_target['전국 관객수']<1000000]),
    len(train_target[(train_target['전국 관객수']>1000000) & (train_target['전국 관객수']<1400000)]),
    len(train_target[(train_target['전국 관객수']>1400000) & (train_target['전국 관객수']<2000000)]),
    len(train_target[(train_target['전국 관객수']>2000000) & (train_target['전국 관객수']<3000000)]),
    len(train_target[(train_target['전국 관객수']>3000000) & (train_target['전국 관객수']<5000000)]),
    len(train_target[train_target['전국 관객수']>5000000])
)

In [ ]:
# movie_resize_분류['코로나'] = np.where((movie_resize_분류['연도'].values >=2020) & (movie_resize_분류['연도'].values <=2022) , 1, 0)

import numpy as np

train_target['분류클래스'] = np.where((train_target['전국 관객수'].values<1000000), 1, 
                                 np.where((train_target['전국 관객수'].values>1000000) & (train_target['전국 관객수'].values<1400000), 2,
                                          np.where((train_target['전국 관객수'].values>1400000) & (train_target['전국 관객수'].values<2000000), 3,
                                                   np.where((train_target['전국 관객수'].values>2000000) & (train_target['전국 관객수'].values<3000000), 4,
                                                            np.where((train_target['전국 관객수'].values>3000000) & (train_target['전국 관객수'].values<5000000), 5, 6))))
)
train_target['분류클래스'].unique()
train_target[['전국 관객수','분류클래스']]

In [ ]:
val_target['분류클래스'] = np.where((val_target['전국 관객수'].values<1000000), 1, 
                                 np.where((val_target['전국 관객수'].values>1000000) & (val_target['전국 관객수'].values<1400000), 2,
                                          np.where((val_target['전국 관객수'].values>1400000) & (val_target['전국 관객수'].values<2000000), 3,
                                                   np.where((val_target['전국 관객수'].values>2000000) & (val_target['전국 관객수'].values<3000000), 4,
                                                            np.where((val_target['전국 관객수'].values>3000000) & (val_target['전국 관객수'].values<5000000), 5, 6))))
)
val_target['분류클래스'].unique()
# val_target[['전국 관객수','분류클래스']]

In [ ]:
test_target['분류클래스'] = np.where((test_target['전국 관객수'].values<1000000), 1, 
                                 np.where((test_target['전국 관객수'].values>1000000) & (test_target['전국 관객수'].values<1400000), 2,
                                          np.where((test_target['전국 관객수'].values>1400000) & (test_target['전국 관객수'].values<2000000), 3,
                                                   np.where((test_target['전국 관객수'].values>2000000) & (test_target['전국 관객수'].values<3000000), 4,
                                                            np.where((test_target['전국 관객수'].values>3000000) & (test_target['전국 관객수'].values<5000000), 5, 6))))
)
test_target['분류클래스'].unique()
# test_target[['전국 관객수','분류클래스']]

In [ ]:
# # 값 스케일링

# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss.fit(train_input)
# train_scaled = ss.transform(train_input)
# test_scaled = ss.transform(test_input)

# 딥러닝 

In [ ]:
# # 딥러닝 층 여기가 중요

# X = tf.keras.layers.Input(shape=[10])

# H = tf.keras.layers.Dense(128)(X)
# H = tf.keras.layers.BatchNormalization()(H)
# H = tf.keras.layers.Activation(tf.nn.swish)(H)

# H = tf.keras.layers.Dense(128)(X)
# H = tf.keras.layers.BatchNormalization()(H)
# H = tf.keras.layers.Activation(tf.nn.swish)(H)

# H = tf.keras.layers.Dense(128)(H)
# H = tf.keras.layers.BatchNormalization()(H)
# H = tf.keras.layers.Activation(tf.nn.swish)(H)
# dropout = tf.keras.layers.Dropout(rate=0.3)(H)

# Y = tf.keras.layers.Dense(1)(dropout)
# model = tf.keras.models.Model(X,Y)
# sgd = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.8,beta_2=0.85)
# # sgd = tf.keras.optimizers.SGD(learning_rate=0.1,momentum=0.7)
# model.compile(optimizer=sgd,loss='mse')

In [ ]:
# 딥러닝 층 여기가 중요

X = tf.keras.layers.Input(shape=[10])

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization(epsilon=1e-5)(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(X)
H = tf.keras.layers.BatchNormalization(epsilon=1e-5)(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

H = tf.keras.layers.Dense(128)(H)
H = tf.keras.layers.BatchNormalization(epsilon=1e-5)(H)
H = tf.keras.layers.Activation(tf.nn.swish)(H)

dropout = tf.keras.layers.Dropout(rate=0.2)(H)
Y = tf.keras.layers.Dense(1)(dropout)

model = tf.keras.models.Model(X,Y)
sgd = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.8,beta_2=0.85)
# sgd = tf.keras.optimizers.SGD(learning_rate=0.01,momentum=0.7)

model.compile(optimizer=sgd,loss='mse')

In [ ]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[9]),
    
    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.BatchNormalization(epsilon=1e-5),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(128, activation='swish'),
    tf.keras.layers.BatchNormalization(epsilon=1e-5),
    tf.keras.layers.Dropout(0.2),

    tf.keras.layers.Dense(1, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# 훈련

history = model.fit(train_input,train_target,batch_size=128,epochs=100,validation_data=(test_input,test_target))

In [ ]:
# 확인용도
model.predict(train_input[:5])

In [ ]:
# 여기랑 비슷한지 보기
train_target[:5]